20221024-173245


## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys
import re
import math
from datetime import datetime
import time
from collections import OrderedDict
from functools import partial
sys.dont_write_bytecode = True
from IPython.core.debugger import set_trace

In [ ]:
import json
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from pathlib import Path
from typing import Any,List, Set, Dict, Tuple, Optional, Iterable, Mapping, Union, Callable, TypeVar

from pprint import pprint
from tqdm import tqdm

In [ ]:
# import for webcrawling
import bs4
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse
import requests

## Import ReprLearn and TileMani packages

In [ ]:
import reprlearn as rl

In [ ]:
from reprlearn.visualize.utils import get_fig, show_timg, show_timgs, show_npimgs, show_batch, make_grid_from_tensors
from reprlearn.utils.misc import info, now2str, today2str, get_next_version_path, n_iter_per_epoch


## Path to data root dirs
- DATA_ROOT: 
  - Use '/data/hayley-old/Tenanbaum2000/data' for MNIST, Mono-MNIST, Rotated-MNIST, Teapots
  - Use `/data/hayley-old/maptiles_v2' folder for Maptile dataset
  

In [ ]:
# ROOT = Path("/data/hayley-old/Tenanbaum2000")
# DATA_ROOT = Path("/data/hayley-old/Tenanbaum2000/data")
# MAP_DATA_ROOT = Path("/data/hayley-old/maptiles_v2/")

data_root = Path("/data/datasets")
real_data_dir = Path("/data/datasets/reverse-eng-data/originals/CelebA/img_align_celeba")
print("data_root: ", str(data_root))
print("real_data_root  (original celeba): ", str(real_data_dir))
print(real_data_dir.exists())

## Helpers

In [ ]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [ ]:
# test: 
urlparse(url) 
# netloc: domain name
# scheme: protocol

## Playground

In [ ]:
class Report:
    def __init__(self, vid: int):
        self.vid = vid
        self.report_url = f'https://confluence.org/confluence.php?visitid={vid}'
        self.img_page_url = f'https://confluence.org/photo.php?visitid={vid}&pic=ALL'
        self.lat = None
        self.lng = None
        self.visit_idx = None
        self.img_urls = []
        self.text = ''
        self.year = None
        self.country = ''
        
        self.update_metadata()

    
    def update_metadata(self) -> None:
        #parse html: get last,lng,text descr, year,country
        # get hrml content of url
        r = requests.get(self.report_url)
        soup = bs4.BeautifulSoup(r.text, 'html.parser')

        # parse html 
#         lat = 
        pass
    
    def download_imgs(self, out_dir: Path) -> None:
        """Effect: sets self.img_urls, download image to out_dir
        """
        pass
    def download_imgs(self,
                  out_dir: Path,
                 filter_func: Optional[Callable]=None, 
                 verbose: bool=False) -> None:
        """Effect: sets self.img_urls, download image to out_dir
        """
        # get hrml content of url
        r = requests.get(self.img_page_url)
        # parse html 
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        

        
        img_tags = soup.find_all("img")

        img_urls = [] 
        for img_tag in img_tags:
            img_alt = img_tag.attrs.get("alt")
            img_url = img_tag.attrs.get("src")
            if not img_url:
                continue

            # concat to get full address to the image
            img_url = urljoin(url, img_url)
            # remove error-prone http get kvey-values in the url
            try:
                pos = img_url.index("?")
                img_url = img_url[:pos]
            except ValueError:
                pass

            # check if img_url is valid and pass filter_func
            if filter_func is not None and not filter_func(img_url, img_alt):
                breakpoint()
                print(f'{img_url} does not pass the filter function') #todo: write to a log file
                continue

            # download img_url
            suffix = Path(img_url).suffix
            filename = self.create_filename(img_alt, suffix) #{vid}-{lat}-{lon}-{direction}.jpg'
            print('Downlaoding: ', img_url)
            download(img_url, out_dir, filename)

            # collect all img_urls
            img_urls.append(img_url)
            if verbose:
                print(f'\n {img_url}')

        self.img_urls = img_urls
        print('Done downloading imgs for ', self.vid)

    
    
    def create_filename(self, img_alt: str, suffix: str) -> str:
#         img_alt = img_alt.lower().replace(' ', '-')
        return f'{self.vid}-{self.lat}-{self.lng}-{img_alt}{suffix}' 
        
        
        

In [ ]:
url = 'https://confluence.org/confluence.php?visitid=22761'


# content of url
r = requests.get(url)

In [ ]:
imgs = soup.find_all("img")

In [ ]:
def get_img_urls(url: str, 
                 filter_func: Optional[Callable]=None, 
                 verbose: bool=False) -> List:
    # content of url
    r = requests.get(url)
    # parse html 
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    imgs = soup.find_all("img")
    
    img_urls = [] 
    for img_tag in imgs:
        img_alt = img_tag.attrs.get("alt")
        img_url = img_tag.attrs.get("src")
        if not img_url:
            continue

        # concat to get full address to the image
        img_url = urljoin(url, img_url)
        # remove error-prone http get kvey-values in the url
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # check if img_url is valid and pass filter_func
        if not is_valid(img_url):
            continue
        if filter_func is not None and not filter_func(img_url):
            print('img_url does not pass the filter function')
            continue

        img_urls.append(img_url)
        if verbose:
            print()
            print(img_tag)
            print(img_url)
    return img_urls

    

In [ ]:
def download(url: str, dirpath: Path, out_bn: Optional[str]=None):
    """Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not dirpath.exists():
        dirpath.mkdir(parents=True)
        print('Created: ', dirpath)
        
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    out_bn = out_bn or  url.split("/")[-1]
    out_fn = dirpath / out_bn
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {out_fn}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(out_fn, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

In [ ]:
def create_filename(lat:str, lng: str, vid: str, img_alt:str, suffix: str):
    img_alt = img_alt.lower().replace(' ', '-')
    return f'{vid}-{lat}-{lng}-{img_alt}{suffix}'
    

In [ ]:
def download_imgs(url: str, 
                  out_dir: Path,
                 filter_func: Optional[Callable]=None, 
                 verbose: bool=False) -> List:
    # content of url
    r = requests.get(url)
    # parse html 
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    img_tags = soup.find_all("img")
    
    img_urls = [] 
    for img_tag in img_tags:
        img_alt = img_tag.attrs.get("alt")
        img_url = img_tag.attrs.get("src")
        if not img_url:
            continue

        # concat to get full address to the image
        img_url = urljoin(url, img_url)
        # remove error-prone http get kvey-values in the url
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        
        # check if img_url is valid and pass filter_func
        if filter_func is not None and not filter_func(img_url, img_alt):
            breakpoint()
            print(f'{img_url} does not pass the filter function') #todo: write to a log file
            continue

        # download img_url
        lat, lng = '10', '11' # temp
        suffix = Path(img_url).suffix
        filename = create_filename(lat, lng, visit_idx, img_alt, suffix) #todo: should includ image format/suffix. e.g.: '{lat}-{lon}-{direction}.png'
        print('Downlaoding: ', img_url)
        download(img_url, out_dir, filename)
        
        # collect all img_urls
        img_urls.append(img_url)
        if verbose:
            print(f'\n {img_url}')
    
            
    return img_urls

    

In [ ]:
# define a valid img_url
def is_valid_img_url(img_url: str, img_alt: str) -> bool:

    # conditions to meet
    meets_cond1 = is_valid(img_url)
    meets_cond2 = '.gif' not in img_url.lower()
    meets_cond3 = 'gps' not in img_alt.lower()
    
    return meets_cond1 and meets_cond2 and meets_cond3


In [21]:
# test:
url = 'https://confluence.org/photo.php?visitid=22761&pic=ALL'
download_imgs(url, out_dir=Path('./temp'), filter_func=is_valid_img_url, verbose=True)


https://confluence.org/graphics/earthsmall2.gif does not pass the filter function
Downlaoding:  https://confluence.org/ru/all/n53e051v2/%70%69%63%31.jpg



 https://confluence.org/ru/all/n53e051v2/%70%69%63%31.jpg
Downlaoding:  https://confluence.org/ru/all/n53e051v2/%70%69%63%32.jpg



 https://confluence.org/ru/all/n53e051v2/%70%69%63%32.jpg
Downlaoding:  https://confluence.org/ru/all/n53e051v2/%70%69%63%33.jpg



 https://confluence.org/ru/all/n53e051v2/%70%69%63%33.jpg
Downlaoding:  https://confluence.org/ru/all/n53e051v2/%70%69%63%34.jpg



 https://confluence.org/ru/all/n53e051v2/%70%69%63%34.jpg
https://confluence.org/ru/all/n53e051v2/%70%69%63%35.jpg does not pass the filter function
Downlaoding:  https://confluence.org/ru/all/n53e051v2/%70%69%63%36.jpg



 https://confluence.org/ru/all/n53e051v2/%70%69%63%36.jpg


['https://confluence.org/ru/all/n53e051v2/%70%69%63%31.jpg',
 'https://confluence.org/ru/all/n53e051v2/%70%69%63%32.jpg',
 'https://confluence.org/ru/all/n53e051v2/%70%69%63%33.jpg',
 'https://confluence.org/ru/all/n53e051v2/%70%69%63%34.jpg',
 'https://confluence.org/ru/all/n53e051v2/%70%69%63%36.jpg']